In [ ]:
import pandas as pd


In [ ]:
# df= pd.read_csv(r'D:\Coding\Python\Algorithm\data\苏州数据\合并后.csv')
# 统计缺失率
# statistic=df.isnull().sum(axis=0)/df.shape[0]
# statistic.to_csv(r'D:\Coding\Python\Algorithm\data\苏州数据\缺失率.csv')
# 保存便于处理的文件
# df=df[['病案号','验光时间']]
# df.to_csv(r'D:\Coding\Python\Algorithm\data\苏州数据\病案号验光时间.csv',index=False)

df = pd.read_csv(r'D:\Coding\Python\Algorithm\data\苏州数据\病案号验光时间.csv')
df['验光时间'] = pd.to_datetime(df['验光时间'])
group = df.groupby('病案号')
count = group.count()
count.rename(columns={'验光时间': '次数'}, inplace=True)
interval = group.max() - group.min()
interval.rename(columns={'验光时间': '最大间隔_天'}, inplace=True)
interval['最大间隔_天'] = interval['最大间隔_天'].apply(lambda x: x.days)
new_df = pd.concat([count, interval], axis=1)
new_df['最大间隔_月'] = new_df['最大间隔_天'] // 30
new_df['平均间隔_天'] = new_df['最大间隔_天'] / (new_df['次数'] - 1)
new_df['平均间隔_月'] = new_df['最大间隔_月'] / (new_df['次数'] - 1)
new_df.to_csv(r'D:\Coding\Python\Algorithm\data\苏州数据\病案号验光时间统计.csv')


In [ ]:
# 处方被多少人用过
chu_fang_df = pd.read_excel(r'D:\Coding\Python\Algorithm\data\苏州数据\处方.xlsx', engine='openpyxl')
chu_fang_df = chu_fang_df[['病案号', '处方']].drop_duplicates()
count = chu_fang_df.groupby('处方').count()
count.rename(columns={'病案号': '使用人数'}, inplace=True)
count.to_csv(r'D:\Coding\Python\Algorithm\data\苏州数据\处方使用人数统计.csv')

In [ ]:
# 使用过有效和无效处方的人
chu_fang_df = pd.read_excel(r'D:\Coding\Python\Algorithm\data\苏州数据\处方.xlsx', engine='openpyxl')
chu_fang_useless = [
    '七叶洋地黄双苷滴眼液', '复方托吡卡胺滴眼液', '托吡卡胺滴眼液',
    '甲钴胺片', '阿托品眼用凝胶', '夏天无滴眼液', '硫酸阿托品眼用凝胶',
    '甲钴胺针', '甲钴胺注射液']
chu_fang_useful = [
    '阿托品滴眼液', '阿托品眼膏', '复方消旋山莨菪碱滴眼液', '消旋山莨菪碱滴眼液', '硫酸阿托品注射液',
    '阿托品针', '硫酸阿托品滴眼液0.01%', '盐酸消旋山莨菪碱注射液', '山莨菪碱针', '盐酸山莨菪碱注射液']
chu_fang_df = chu_fang_df[['病案号', '处方']]
chu_fang_useful_df = chu_fang_df[chu_fang_df['处方'].isin(chu_fang_useful)]
chu_fang_useless_df = chu_fang_df[chu_fang_df['处方'].isin(chu_fang_useless)]
chu_fang_useful_df.to_csv(r'D:\Coding\Python\Algorithm\data\苏州数据\处方_useful.csv', index=False)
chu_fang_useless_df.to_csv(r'D:\Coding\Python\Algorithm\data\苏州数据\处方_useless.csv', index=False)

In [ ]:
# 合并大表中有效和无效的人
he_bing_df = pd.read_csv(r'D:\Coding\Python\Algorithm\data\苏州数据\合并后.csv')
he_bing_useful_df = he_bing_df[he_bing_df['病案号'].isin(chu_fang_useful_df['病案号'])]
he_bing_not_useful_df = he_bing_df[~he_bing_df['病案号'].isin(chu_fang_useful_df['病案号'])]
he_bing_useless_df = he_bing_df[he_bing_df['病案号'].isin(chu_fang_useless_df['病案号'])]
he_bing_useful_df.to_csv(r'D:\Coding\Python\Algorithm\data\苏州数据\合并后_useful.csv', index=False)
he_bing_not_useful_df.to_csv(r'D:\Coding\Python\Algorithm\data\苏州数据\合并后_not_useful.csv', index=False)
he_bing_useless_df.to_csv(r'D:\Coding\Python\Algorithm\data\苏州数据\合并后_useless.csv', index=False)


In [ ]:
# 月进展率
def month_rate_of_progress(x: pd.DataFrame):
    diff = x.sort_values(by='随访时间').diff(periods=1, axis=0)
    diff['随访时间'] = diff['随访时间'].apply(lambda x: x.days)
    new_df = pd.DataFrame(columns=['月进展率'])
    new_df['月进展率'] = diff['SE'] / (diff['随访时间'] // 30)
    return new_df


# 先去掉没有用的数据
# df = pd.read_excel(r'D:\Coding\Python\Algorithm\data\age_result.xlsx', sheet_name='Sheet1', engine='openpyxl')
# df=df[['姓名', '出生年月', '随访时间', '主觉验光SE（左）','主觉验光SE（右）']]
# df.to_excel(r'D:\Coding\Python\Algorithm\data\data.xlsx',engine='openpyxl',index=False)
# 重新读取
df = pd.read_excel(r'D:\Coding\Python\Algorithm\data\data.xlsx', sheet_name='Sheet1', engine='openpyxl')

df_left = df[['姓名', '出生年月', '随访时间', '主觉验光SE（左）']].dropna().rename(columns={'主觉验光SE（左）': 'SE'})
df_left = df_left.groupby(['姓名', '出生年月']).filter(lambda x: x['SE'].count() > 1)
df_left = df_left.groupby(['姓名', '出生年月']).apply(month_rate_of_progress)
df_left.to_excel(r'D:\Coding\Python\Algorithm\data\left.xlsx', engine='openpyxl')

df_right = df[['姓名', '出生年月', '随访时间', '主觉验光SE（右）']].dropna().rename(columns={'主觉验光SE（右）': 'SE'})
df_right = df_right.groupby(['姓名', '出生年月']).filter(lambda x: x['SE'].count() > 1)
df_right = df_right.groupby(['姓名', '出生年月']).apply(month_rate_of_progress)
df_right.to_excel(r'D:\Coding\Python\Algorithm\data\right.xlsx', engine='openpyxl')